# LSTM/GRU/MLP univariate/multivariate

### Create 2d array

In [13]:
import numpy as np
import pandas as pd
from datetime import datetime

# Choose frequency, binsize, longitude, latitude
freq = 'w'
binsize = 2
longitude_W = 134 # minimum is 134
longitude_E = 174 # maximum is 174
latitude_S = 10 # minimum is 10
latitude_N = 60 # minimum is 60

# load earthquake data for defined area
data = pd.read_csv('data/Japan_10_60_134_174_1973_2023_V2.csv', index_col=0)
data['Time'] = pd.to_datetime(data.Time)
data = data[(data.Longitude >= longitude_W) & (data.Longitude <= longitude_E) & (data.Latitude >= latitude_S) & (data.Latitude <= latitude_N)]
data.set_index('Time', inplace=True)
df = data.sort_index()
df['timestamp'] = df.index.map(pd.Timestamp)
df['unix_timestamp'] = df['timestamp'].dt.floor('s').astype(int)
df

,Longitude,Latitude,Magnitude,Depth,timestamp,unix_timestamp
Time,,,,,,
1973-01-03 06:47:47.800,145.6740,16.8090,4.0,177.000,1973-01-03 06:47:47.800,94891667000000000
1973-01-05 05:31:05.800,140.8660,33.4750,4.5,56.000,1973-01-05 05:31:05.800,95059865000000000
1973-01-05 11:48:37.500,140.9150,33.1600,3.9,33.000,1973-01-05 11:48:37.500,95082517000000000
1973-01-05 13:39:50.400,146.8230,48.1620,5.1,430.000,1973-01-05 13:39:50.400,95089190000000000
1973-01-06 09:58:32.900,144.5900,12.2760,4.7,32.000,1973-01-06 09:58:32.900,95162312000000000
...,...,...,...,...,...,...
2022-12-30 22:42:08.690,160.3568,52.4068,4.7,10.000,2022-12-30 22:42:08.690,1672440128000000000
2022-12-31 03:27:38.307,146.1845,19.1089,4.0,89.055,2022-12-31 03:27:38.307,1672457258000000000
2022-12-31 15:24:22.787,143.7703,12.1600,4.7,10.000,2022-12-31 15:24:22.787,1672500262000000000


### Splitting the data

In [14]:
from sklearn.model_selection import train_test_split

# split data in train, val, test

# Activate when only forecasting one region
# matrix = np.reshape(matrix, (matrix.shape[0], matrix.shape[1], 1))

train, val_test = train_test_split(df, test_size=.3, shuffle=False, random_state=43)
val, test = train_test_split(val_test, test_size=.5, shuffle=False, random_state=43)

### Generate datasets from timeseries

In [ ]:
from tensorflow.keras.preprocessing import timeseries_dataset_from_array
import tensorflow as tf

def dataset_generator(data, seq_length, cutoff):

  input_data = data # data[:-seq_length]
  targets = data[seq_length:]
  dataset = timeseries_dataset_from_array(input_data, (targets >= cutoff).astype(int), sequence_length=seq_length, sampling_rate=1, sequence_stride=1, shuffle=False, batch_size=len(data))
  """
  for batch in dataset:
    inputs, targets = batch
    assert np.array_equal(inputs[0], data[:seq_length])  # First sequence: steps [0-9]
    assert np.array_equal(targets[0], data[seq_length])  # Corresponding target: step 10
    """
  return dataset

# Set lookback timewindow
timewindow = 365
cutoff = 4.5

train_dataset = dataset_generator(train, timewindow, cutoff)
val_dataset = dataset_generator(val, timewindow, cutoff)
test_dataset = dataset_generator(test, timewindow, cutoff)

# Create train set
for batch in train_dataset:
    X_train, y_train = batch

# y_train = tf.reshape(y_train, shape=[y_train.shape[0], 1, y_train.shape[1]])

# Collapse the depth dimension and converts all non-zero values to 1 and zero values to 0
# y_train = tf.cast(tf.reduce_max(y_train, axis=2, keepdims=True) > 0, dtype=tf.int32)

# Create validation set
for batch in val_dataset:
    X_val, y_val = batch

# y_val = tf.reshape(y_val, shape=[y_val.shape[0], 1, y_val.shape[1]])

# Collapse the depth dimension and converts all non-zero values to 1 and zero values to 0
# y_val = tf.cast(tf.reduce_max(y_val, axis=2, keepdims=True) > 0, dtype=tf.int32)

# Create test set
for batch in test_dataset:
    X_test, y_test = batch

# y_test = tf.reshape(y_test, shape=[y_test.shape[0], 1, y_test.shape[1]])

# Collapse the depth dimension and converts all non-zero values to 1 and zero values to 0
# y_test = tf.cast(tf.reduce_max(y_test, axis=2, keepdims=True) > 0, dtype=tf.int32)

################################# Use for MLP
# Flatten 1 and 2 dimensions of X's for multivariate MLP
# X_train = np.reshape(X_train, (X_train.shape[0],-1))
# X_val = np.reshape(X_val, (X_val.shape[0],-1))
# X_test = np.reshape(X_test, (X_test.shape[0],-1))

X_test.shape, y_test.shape

In [16]:
from tensorflow.keras.preprocessing import timeseries_dataset_from_array
import tensorflow as tf

def dataset_generator(data, seq_length, cutoff):

  input_data = data
  dataset = timeseries_dataset_from_array(input_data, targets=None, sequence_length=seq_length, sampling_rate=1, sequence_stride=1, shuffle=False, batch_size=len(data))

  return dataset

# Set lookback timewindow
lookback_window = 52
cutoff = 4.5
future_timesteps = 4
timewindow = lookback_window + future_timesteps

train_dataset = dataset_generator(train, timewindow, cutoff)
val_dataset = dataset_generator(val, timewindow, cutoff)
test_dataset = dataset_generator(test, timewindow, cutoff)

# Create train set
for batch in train_dataset:
    X_train = batch[:,:-future_timesteps,:]
    y_train = batch[:,-future_timesteps:,:] >= cutoff
    y_train = np.max(y_train, axis=1)
    y_train = np.reshape(y_train, (y_train.shape[0], 1, y_train.shape[1])).astype(int)

# y_train = tf.reshape(y_train, shape=[y_train.shape[0], 1, y_train.shape[1]])

# Collapse the depth dimension and converts all non-zero values to 1 and zero values to 0
# y_train = tf.cast(tf.reduce_max(y_train, axis=2, keepdims=True) > 0, dtype=tf.int32)

# Create validation set
for batch in val_dataset:
    X_val = batch[:,:-future_timesteps,:]
    y_val = batch[:,-future_timesteps:,:] >= cutoff
    y_val = np.max(y_val, axis=1)
    y_val = np.reshape(y_val, (y_val.shape[0], 1, y_val.shape[1])).astype(int)

# y_val = tf.reshape(y_val, shape=[y_val.shape[0], 1, y_val.shape[1]])

# Collapse the depth dimension and converts all non-zero values to 1 and zero values to 0
# y_val = tf.cast(tf.reduce_max(y_val, axis=2, keepdims=True) > 0, dtype=tf.int32)

# Create test set
for batch in test_dataset:
    X_test = batch[:,:-future_timesteps,:]
    y_test = batch[:,-future_timesteps:,:] >= cutoff
    y_test = np.max(y_test, axis=1)
    y_test = np.reshape(y_test, (y_test.shape[0], 1, y_test.shape[1])).astype(int)

# y_test = tf.reshape(y_test, shape=[y_test.shape[0], 1, y_test.shape[1]])

# Collapse the depth dimension and converts all non-zero values to 1 and zero values to 0
# y_test = tf.cast(tf.reduce_max(y_test, axis=2, keepdims=True) > 0, dtype=tf.int32)

################################# Use for MLP
# Flatten 1 and 2 dimensions of X's for multivariate MLP
# X_train = np.reshape(X_train, (X_train.shape[0],-1))
# X_val = np.reshape(X_val, (X_val.shape[0],-1))
# X_test = np.reshape(X_test, (X_test.shape[0],-1))

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).

## Construct LSTM (univariate/multivariate)

In [ ]:
from keras.models import Sequential, Model
from keras.layers import LSTM, Dense, Flatten, Input, TimeDistributed, Dropout, RepeatVector, BatchNormalization
from keras.layers import LSTM, GRU, Bidirectional, SimpleRNN
from tensorflow.keras.utils import plot_model
import pandas as pd
import numpy as np
import keras
from keras.callbacks import EarlyStopping
from keras import regularizers
from keras import backend as K

keras.backend.clear_session()

# define model
model = Sequential()

model.add(BatchNormalization())
model.add(LSTM(512, activation='LeakyReLU',
               return_sequences=False,
               input_shape=(X_train.shape[1], X_train.shape[2]),
               kernel_regularizer=regularizers.L1L2(l1=1e-4, l2=1e-4),
               bias_regularizer=regularizers.L1L2(l1=1e-4, l2=1e-4),
               # dropout=0.4,
               ))
model.add(RepeatVector(y_train.shape[1]))
model.add(BatchNormalization())

model.add(LSTM(512, activation='LeakyReLU', return_sequences=True))
model.add(BatchNormalization())
model.add(TimeDistributed(Dense(X_train.shape[2], activation="sigmoid")))
"""
model.add(Dense(12, input_dim=(X_train.shape[1]), activation='relu'))
model.add(BatchNormalization())
model.add(Dense(12, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(12, activation='relu'))
model.add(BatchNormalization())
"""

# model.add(Dense(1024, activation='LeakyReLU'))
# model.add(Dense(80, activation='sigmoid'))

opt = keras.optimizers.Adam(learning_rate=0.001)
loss = tf.keras.losses.BinaryFocalCrossentropy( apply_class_balancing=True, alpha=.8, gamma=1)
model.compile(optimizer=opt, loss=loss, metrics=[keras.metrics.Precision(), keras.metrics.Recall()])
model.build(input_shape=X_train.shape)
model.summary()


### fit model

In [ ]:
# early stopping
callback = EarlyStopping(monitor='val_loss', patience=5)
# fit model
history = model.fit(x=X_train,
                    y=y_train,
                    validation_data=(X_val,y_val),
                    batch_size=128,
                    epochs=100,
                    verbose=1,
                    callbacks=[callback],
                    shuffle=False)

### predict

In [ ]:
y_pred = model.predict(X_test)

# y_pred = scaler.inverse_transform(y_pred)
# y_test = scaler.inverse_transform(test)

### evaluation metrics

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(np.array(y_test).flatten(), y_pred.flatten() >= .5)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(np.array(y_test).flatten(), y_pred.flatten() >= .5)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(np.array(y_test).flatten(), y_pred.flatten() >= .5)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(np.array(y_test).flatten(), y_pred.flatten() >= .5)
print('F1 score: %f' % f1)

class_names = ['M<4.5', 'M>=4.5']

print(classification_report(np.array(y_test).flatten(), y_pred.flatten() >= .5, target_names=class_names))

# Calculate the proportion of the majority per row, column combination over all batches
majority_prop = np.mean(test >= cutoff, axis=0)[:]

# Calculate the complement for values lower than 0.5
majority_prop = np.where(majority_prop < 0.5, 1 - majority_prop, majority_prop)
zeroR = majority_prop.mean()

print("zeroR:", round(zeroR,4))

### confusion matrix

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
sns.set(rc={'figure.figsize':(5,5)})
p = sns.heatmap(confusion_matrix(np.array(y_test).flatten(), y_pred.flatten() >= .5), annot=True, fmt='g')
p.set_xlabel("Predicted")
p.set_ylabel("True")
p.xaxis.set_ticklabels(['M<4.5', 'M>=4.5'], ha="center", va="center")
p.yaxis.set_ticklabels(['M<4.5', 'M>=4.5'], rotation=0, va="center")

### ROC curve

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

def plot_ROC_AUC(X,y):

    # Use the trained model to predict the class probabilities for the validation set
    y_prob = model.predict(X)
    # y_pred = scaler.inverse_transform(y_prob)
    # y_test = scaler.inverse_transform(y)

    # Compute micro-average ROC curve and ROC area
    fpr, tpr, _ = roc_curve(np.array(y_test).flatten(), y_pred.flatten())
    roc_auc = auc(fpr, tpr)

    # Plot micro-average ROC curve
    plt.figure(figsize=(5,5))
    plt.plot(fpr, tpr, label='ROC curve (area = {0:0.2f})'
            ''.format(roc_auc), color='blue', linewidth=1)

    plt.plot([0, 1], [0, 1], 'k--', linewidth=1)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend(loc="lower right")
    # plt.savefig(savefig)
    plt.show()

plot_ROC_AUC(X_test, y_test)

### Precision-Recall curve

In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc, f1_score
import matplotlib.pyplot as plt

precision, recall, _ = precision_recall_curve(np.array(y_test).flatten(), y_pred.flatten())
auc = auc(recall, precision)

# Calculate the proportion of the majority per row, column combination over all batches
majority_prop = np.mean(test >= cutoff, axis=0)[:]

# Calculate the complement for values lower than 0.5
majority_prop = np.where(majority_prop < 0.5, 1 - majority_prop, majority_prop)
zeroR = majority_prop.mean()

# plot the precision-recall curves
no_skill = (np.array(y_test).flatten() == 1).sum() / len(np.array(y_test).flatten())
plt.plot([0, 1], [no_skill,no_skill], linestyle='--', label='class 1 = %.3f' % (no_skill))
plt.plot(recall, precision, label='PR_curve, AUC = %.3f' % (auc))
# axis labels
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('PR Curve')
# show the legend
plt.legend()
# show the plot
plt.show()

### Plot predicted against true

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Reshape data to grid
y_pred = np.reshape(y_pred, (y_pred.shape[0], tensor.shape[1], tensor.shape[2]))
y_test= np.reshape(y_test, (y_test.shape[0], tensor.shape[1], tensor.shape[2]))

# Choose timesteps to plot
timestep = 1


# Extract the data for the chosen timesteps from the tensor
data1 = y_pred[timestep, :, :] >= 0.5
data2 = y_test[timestep, :, :]

# Create a figure with two subplots
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(9, 4.5))

# Plot the data in each subplot
sns.heatmap(data1, cmap='viridis', vmin=0, vmax=1, linewidths=0.5, linecolor='grey', annot=False, ax=ax1)
sns.heatmap(data2, cmap='viridis', vmin=0, vmax=1, linewidths=0.5, linecolor='grey', annot=False, ax=ax2)

# Set the plot titles and axis labels
ax1.set_title(f'Predicted {cutoff} magnitude at timestep {timestep}')
ax1.set_xlabel('Longitude bin')
ax1.set_ylabel('Latitude bin')

ax2.set_title(f'True {cutoff} magnitude at timestep {timestep}')
ax2.set_xlabel('Longitude bin')
ax2.set_ylabel('Latitude bin')

# Show the plot
plt.show()